In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import pulp
from mymodule import *
from pulp_func import *

df2023_result = pd.read_excel('../excels/附件3/2023_result.xlsx')
# 去除列名末尾的空格。很奇怪
df2023_result.columns = [column.rstrip(' ') for column in df2023_result.columns]
print(df2023_result.columns)
df2024_result = pd.read_excel('../excels/附件3/2024_result.xlsx')
# 去除列名末尾的空格。很奇怪
df2024_result.columns = [column.rstrip(' ') for column in df2024_result.columns]
# print(df2024_result.columns)

Index(['种植季次', '种植地块', '黄豆', '黑豆', '红豆', '绿豆', '爬豆', '小麦', '玉米', '谷子', '高粱',
       '黍子', '荞麦', '南瓜', '红薯', '莜麦', '大麦', '水稻', '豇豆', '刀豆', '芸豆', '土豆', '西红柿',
       '茄子', '菠菜', '青椒', '菜花', '包菜', '油麦菜', '小青菜', '黄瓜', '生菜', '辣椒', '空心菜',
       '黄心菜', '芹菜', '大白菜', '白萝卜', '红萝卜', '榆黄菇', '香菇', '白灵菇', '羊肚菌'],
      dtype='object')


In [2]:
result1 = df21.groupby(['作物编号', '地块类型', '种植季次'])['种植面积/亩'].sum().reset_index()
result1.columns = ['作物编号', '地块类型', '种植季次', '种植面积/亩']
merged_df = pd.merge(result1, df22[['作物编号', '作物名称','作物类型' , '地块类型', '种植季次', '亩产量/斤', '种植成本/(元/亩)', '销售单价/(元/斤)']], on=['作物编号', '种植季次', '地块类型'], how='left')
merged_df['单地块类型总产量/斤'] = merged_df['种植面积/亩'] * merged_df['亩产量/斤']
merged_df['销售单价平均值'] = merged_df['销售单价/(元/斤)'].apply(calculate_average_price)
merged_df['利润'] = (merged_df['销售单价平均值'] * merged_df['单地块类型总产量/斤']) - (merged_df['种植成本/(元/亩)'] * merged_df['种植面积/亩'])
merged_df['每亩利润'] = merged_df['利润'] / merged_df['种植面积/亩']
# print(merged_df)
total_profit_2023 = merged_df.groupby(['作物编号', '作物名称'])[['利润', '每亩利润']].sum().reset_index()
total_profit_2023.columns = ['作物编号', '作物名称', '总利润/元', '每亩总利润/元']
# print(total_profit)
total_yield_2023 = merged_df.groupby(['作物编号', '作物名称'])['单地块类型总产量/斤'].sum().reset_index()
total_yield_2023.columns = ['作物编号', '作物名称', '总产量/斤']
# print(total_yield_2023)

In [3]:
fields = create_fields(df11)
# for field in fields:
#     print(field)
new_fields = create_new_fields(fields)
# for field in new_fields:
#     print(field)
crops = create_crops(df12, df22)
# for crop in crops:
#     print(crop)    

In [4]:
variables, binary_variables, actual_sales, excess_yield = create_variables(new_fields, crops)
# 创建线性规划问题
prob = pulp.LpProblem("Crop_Planting_Optimization", pulp.LpMaximize)
profit = define_objective_function(variables, actual_sales, excess_yield, crops, new_fields)
# 将目标函数添加到问题中
prob += profit
# print(profit)
prob = add_constraints(prob, variables, binary_variables, actual_sales, 
                       excess_yield, crops, new_fields, total_yield_2023, df2024_result, df2023_result)


In [5]:
# 求解问题
prob.solve()

1

In [6]:
print("Status:", pulp.LpStatus[prob.status])
# print("Objective:", pulp.value(prob.objective))

# 打印所有变量及其值
# for v in prob.variables():
#     print(v.name, "=", v.varValue)

# for v in prob.variables():
#     if v.varValue > 0:
#         print(v.name, "=", v.varValue)

# 打印所有约束条件
# for name, constraint in prob.constraints.items():
#     print(f"{name}: {constraint}")

# 打印模型的详细信息
# print(prob)

Status: Optimal


In [7]:
update_new_fields(variables, new_fields)
# 打印结果
# for field in new_fields:
#     print(field.field_name, field.season, field.planted_crop)
df_template = pd.read_excel('../excels/附件3/template.xlsx')
output(df_template, 2025, new_fields)
# print(df_template)

e:\Users\Timothy\anaconda3\Lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")
